<a href="https://colab.research.google.com/github/cleanshafay/open3d_experiments/blob/main/Adding_Noise_and_trying_ICP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Time to see how well ICP performs if we add random noise along with the rotation**
The error could be misleading this time if you apply the inverse to the noisy image so we'll apply the inverse of the rotation matrix found using ICP

In [1]:
!pip install open3D

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


Loading armadillomesh from open3d examples

In [2]:
import open3d as o3d

# Load the armadillo mesh
ArmadilloMesh = o3d.data.ArmadilloMesh()
mesh = o3d.io.read_triangle_mesh(ArmadilloMesh.path)

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/ArmadilloMesh.ply
[Open3D INFO] Downloaded to /root/open3d_data/download/ArmadilloMesh/ArmadilloMesh.ply


In [3]:
mesh.compute_vertex_normals()

TriangleMesh with 172974 points and 345944 triangles.

Reducing the number of points

In [4]:
# Sample points from the mesh
pcd = mesh.sample_points_poisson_disk(number_of_points=1000)

In [5]:
import plotly.graph_objects as go
import numpy as np

# Convert Open3D point cloud to NumPy array
xyz = np.asarray(pcd.points)

# Create a 3D scatter plot
scatter = go.Scatter3d(x=xyz[:, 0], y=xyz[:, 1], z=xyz[:, 2], mode='markers', marker=dict(size=1))
fig = go.Figure(data=[scatter])
fig.show()

In [24]:
# Apply an arbitrary rotation to the original point cloud
R = o3d.geometry.get_rotation_matrix_from_xyz((np.pi / 4, np.pi / 4, np.pi / 4))
rotated_pcd = pcd.rotate(R, center=(0, 0, 0))

In [25]:
# Convert Open3D point cloud to NumPy array
xyz_rot = np.asarray(rotated_pcd.points)

# Create a 3D scatter plot
scatter = go.Scatter3d(x=xyz_rot[:, 0], y=xyz_rot[:, 1], z=xyz[:, 2], mode='markers', marker=dict(size=1.0))
fig = go.Figure(data=[scatter])
fig.show()

Adding noise to the armadillo

In [26]:
# Add Gaussian noise
noise = np.random.normal(loc=0, scale=0.005, size=xyz_rot.shape)
xyz_noisy = xyz_rot + noise

# Update rotated_pcd with noisy data (optional)
noisy_rotated_pcd = o3d.geometry.PointCloud()
noisy_rotated_pcd.points = o3d.utility.Vector3dVector(xyz_noisy)
import plotly.graph_objects as go

scatter = go.Scatter3d(
    x=xyz_noisy[:, 0],
    y=xyz_noisy[:, 1],
    z=xyz_noisy[:, 2],
    mode='markers',
    marker=dict(size=1.0, color='red')
)

fig = go.Figure(data=[scatter])
fig.update_layout(title="Noisy Rotated Point Cloud")
fig.show()


In [28]:
# Two point clouds in one plot
scatter = go.Scatter3d(x=xyz_rot[:, 0], y=xyz_rot[:, 1], z=xyz[:, 2], mode='markers', marker=dict(size=1, color='green'), name="Rotated")

scatter_noisy = go.Scatter3d(x=xyz_noisy[:, 0], y=xyz_noisy[:, 1], z=xyz_noisy[:, 2], mode='markers',
                             marker=dict(size=1, color='red'), name="Noisy Rotated")

fig = go.Figure(data=[scatter, scatter_noisy])
fig.update_layout(title="Rotated vs Noisy Rotated Point Clouds")
fig.show()


# Finding estimated rotations matrix through ***ICP***
Note that this matrix is found using the noisy rotated point cloud but will still work if its inverse is applied to the rotated image

In [37]:
# Use ICP to find the rotation
threshold = 0.02  # Distance threshold
trans_init = np.identity(4)  # Initial guess (identity matrix)
trans_init[:3, :3] = R  # We set the initial rotation to the known rotation
reg_p2p = o3d.pipelines.registration.registration_icp(
    source=noisy_rotated_pcd, target=pcd, max_correspondence_distance=threshold,
    init=trans_init
)

# Extract the rotation matrix from the transformation matrix
estimated_rotation_matrix = reg_p2p.transformation[:3, :3]
rotation_matrix = reg_p2p.transformation[:3, :3]
print("Estimated rotation matrix:")
print(rotation_matrix)

Estimated rotation matrix:
[[ 0.5        -0.5         0.70710678]
 [ 0.85355339  0.14644661 -0.5       ]
 [ 0.14644661  0.85355339  0.5       ]]


Multiplying the inverse of the estimated rotation matrix with the rotated image to check if the image goes back to its original state

In [36]:
# Extract the rotation matrix from the transformation matrix
estimated_rotation_matrix = reg_p2p.transformation[:3, :3]

# Apply the inverse of the estimated rotation to the rotated point cloud
inverse_rotation_matrix = np.linalg.inv(estimated_rotation_matrix)
rotated_back_pcd = rotated_pcd.rotate(inverse_rotation_matrix, center=(0, 0, 0))

# Compare the original point cloud to the one rotated back to its original state
# We can use the mean squared error (MSE) between corresponding points as a metric
original_points = np.asarray(pcd.points)
rotated_back_points = np.asarray(rotated_back_pcd.points)
mse = np.mean(np.linalg.norm(original_points - rotated_back_points, axis=1) ** 2)

# Check if the MSE is below a certain tolerance
tolerance = 1e-6
if mse < tolerance:
    print(f"Test passed: MSE = {mse}")
else:
    print(f"Test failed: MSE = {mse}")

Test passed: MSE = 0.0
